# Test Develop Mock Circuit

In [1]:
import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.mock.cell import Cell

In [2]:
from neuro_dmt.models.bluebrain.circuit.geometry import Position
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

(1.0, 1.0, 1.0)
[1. 1. 1.]
[[ 0.22635212  0.2078533   0.13377548]
 [ 0.06199882 -0.05772724  0.77261675]
 [ 0.93111206  0.89891481 -0.94479981]]


In [3]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")

In [4]:
example_cell.as_dict

{'etype': 'not-defined',
 'layer': 1,
 'morph_class': 'not-defined',
 'mtype': 'L23_BTC',
 'nucleus': 'not-defined',
 'region': 'brain',
 'synapse_class': 'INH',
 'x': 1.0,
 'y': 2.0,
 'z': 3.0}

In [5]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition, circuit_connectivity
circuit_composition.cell_density.head()

error         mean
mtype  layer                    
L1_DAC 1        0.0  3094.333333
       2        0.0     0.000000
       3        0.0     0.000000
       4        0.0     0.000000
       5        0.0     0.000000

In [6]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder

In [7]:
test_circuit_builder =\
    CircuitBuilder(
        composition=circuit_composition,
        connectivity=circuit_connectivity)

In [8]:
test_circuit_builder.get_cell_density( mtype="L1_DAC", layer=1 )

,,error,mean
mtype,layer,,
L1_DAC,1,0.0,3094.333333


In [9]:
print(circuit_composition.cell_density.head())
print(test_circuit_builder.get_number_cells(1, "L1_DAC"))

              error         mean
mtype  layer                    
L1_DAC 1        0.0  3094.333333
       2        0.0     0.000000
       3        0.0     0.000000
       4        0.0     0.000000
       5        0.0     0.000000
23


In [10]:
circuit_cells = test_circuit_builder.get_cells()

In [11]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

number of cells 12237
         etype  layer  morph_class   mtype      nucleus region synapse_class  \
0  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
1  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
2  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
3  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
4  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   

            x            y           z  
0   30.003654  1789.532950  227.640151  
1   55.352221  1781.672151  165.591232  
2  230.980671  1723.316828   90.535777  
3   27.325838  1802.884883  109.308426  
4   68.590453  1753.726865  178.286974  


In [12]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import MockCircuit
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)

In [13]:
sum(len(xs) for xs in mock_circuit.connectome.afferent_adjacency)

24475456

In [ ]:
mock_circuit.